<a href="https://colab.research.google.com/github/varunpenumudi/notes/blob/master/Pytorch_codemy/Simple_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# create a Model that inherits nn.Module

class SimpleModel(nn.Module):
    # Input layer (4 features of flower) --> Hidden Layer H1 --> Hidden Layer H2 --> Output(3 classes of flower)
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features, h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x


In [6]:
# set the random seed manually for Pytorch
torch.manual_seed(41)

# model object that is of SimpleModel Class
model = SimpleModel()